### 🛠️ Khởi tạo Môi trường và Cấu hình Dữ liệu

1.  **Cấu hình Đường dẫn (Path):**
    * Tự động xác định thư mục gốc của dự án.
    * Thêm thư mục `src` vào `sys.path` để có thể import các module tùy chỉnh.

2.  **Import Module Xử lý:**
    * `cleaning_diem_thi`: Chứa các hàm làm sạch dữ liệu và hiển thị báo cáo thống kê.
    * `preprocessing_diem_thi`: Chứa các hàm xử lý vector hóa, thu thập dữ liệu (crawl) và lấy tọa độ địa lý (geocoding).

3.  **Thiết lập Tham số & Tài nguyên:**
    * Định nghĩa đường dẫn đến thư mục `data` và file cấu hình tổ hợp môn (`to_hop.json`).
    * Tải biến môi trường (Mapbox Token) từ file `.env` để sử dụng cho bản đồ.
    * Khởi tạo danh sách các năm cần xử lý dữ liệu (từ **2019 đến 2024**).

In [1]:
import sys
import os
import pandas as pd
from pathlib import Path
import numpy as np
import time
import requests
from IPython.display import display
from dotenv import load_dotenv

# Cấu hình đường dẫn để import từ 'src' 
notebook_dir = Path.cwd()
project_root = notebook_dir.parent
src_dir = project_root / 'src'

if str(src_dir) not in sys.path:
    sys.path.append(str(src_dir))

# IMPORT CÁC HÀM TÙY CHỈNH
try:
    from cleaning_diem_thi import run_full_preprocessing, display_stats_report
    print(" Import các hàm từ 'src' thành công!")
except ImportError as e:
    print(f" LỖI KHÔNG TÌM THẤY MODULE: {e}")
try: 
    from preprocessing_diem_thi import process_files_vectorized, crawl_province, geocode_province
    print(" Import thành công")
except ImportError as e: 
    print(f" LỖI KHÔNG TÌM THẤY MODULE: {e}")

# Cấu hình đường dẫn Data và Config 
DATA_DIR = project_root / 'data'
CONFIG_PATH = DATA_DIR / 'to_hop.json' 
load_dotenv()
MAPBOX_TOKEN = os.getenv('MAPBOX_TOKEN')
# Định nghĩa các tham số
years_to_process = {
    2019: CONFIG_PATH,
    2020: CONFIG_PATH,
    2021: CONFIG_PATH,
    2022: CONFIG_PATH,
    2023: CONFIG_PATH,
    2024: CONFIG_PATH,
}

print(f"Thiết lập thành công. Sẵn sàng xử lý {len(years_to_process)} năm.")

 Import các hàm từ 'src' thành công!
 Import thành công
Thiết lập thành công. Sẵn sàng xử lý 6 năm.


### 🚀 Thực thi Quy trình Tiền xử lý & Làm sạch Dữ liệu

1.  **Thiết lập Ngưỡng lọc (Thresholds):**
    * `NGUONG_DIEM_LIET = 1.0`: Định nghĩa mức điểm liệt (theo quy chế thi, điểm <= 1.0 là điểm liệt).
    * `NGUONG_DIEM_SAN_TO_HOP = 15.0`: Thiết lập mức điểm sàn xét tuyển tổ hợp để lọc ra tập dữ liệu tiềm năng.

2.  **Kích hoạt Xử lý (`run_full_preprocessing`):**
    * Hàm này sẽ duyệt qua danh sách các năm (`years_to_process`).
    * Thực hiện làm sạch, loại bỏ các thí sinh vi phạm quy chế hoặc thiếu dữ liệu.
    * Trả về `all_stats_data`: Chứa các thông số thống kê chi tiết sau khi dữ liệu đã được làm sạch.

In [3]:
NGUONG_DIEM_LIET = 1.0
NGUONG_DIEM_SAN_TO_HOP = 12.5
all_stats_data = run_full_preprocessing(
    years_to_process, 
    DATA_DIR, 
    nguong_diem_liet=NGUONG_DIEM_LIET, 
    nguong_diem_to_hop=NGUONG_DIEM_SAN_TO_HOP
)

Bắt đầu xử lý 6 năm...
Ngưỡng lọc hiện tại: Liệt <= 1.0, Tổ hợp >= 12.5
----------------------------------------
ĐANG XỬ LÝ NĂM: 2019
Đang đọc: diem_thi_toan_quoc_2019.csv
 ĐÃ LƯU: diem_thi_2019_new.csv (Số lượng: 835697)
----------------------------------------
ĐANG XỬ LÝ NĂM: 2020
Đang đọc: diem_thi_toan_quoc_2020.csv
 ĐÃ LƯU: diem_thi_2020_new.csv (Số lượng: 853737)
----------------------------------------
ĐANG XỬ LÝ NĂM: 2021
Đang đọc: diem_thi_toan_quoc_2021.csv
 ĐÃ LƯU: diem_thi_2021_new.csv (Số lượng: 970756)
----------------------------------------
ĐANG XỬ LÝ NĂM: 2022
Đang đọc: diem_thi_toan_quoc_2022.csv
 ĐÃ LƯU: diem_thi_2022_new.csv (Số lượng: 982436)
----------------------------------------
ĐANG XỬ LÝ NĂM: 2023
Đang đọc: diem_thi_toan_quoc_2023.csv
 ĐÃ LƯU: diem_thi_2023_new.csv (Số lượng: 1004991)
----------------------------------------
ĐANG XỬ LÝ NĂM: 2024
Đang đọc: diem_thi_toan_quoc_2024.csv
 ĐÃ LƯU: diem_thi_2024_new.csv (Số lượng: 1054051)
--------------------------

### 📊 Hiển thị Báo cáo Thống kê Dữ liệu lọc

In [4]:
# Báo cáo thông số
display_stats_report(all_stats_data)


BẢNG TỔNG HỢP KẾT QUẢ LỌC DỮ LIỆU


,Số lượng trước khi lọc,Số lượng dữ liệu lỗi (SBD rỗng/trùng lặp),Số lượng thí sinh bị điểm liệt,Số lượng thí sinh tổ hợp không đạt yêu cầu,Số lượng sau khi lọc,Phần % loại bỏ
Năm,,,,,,
2019,882594,0,3671,43226,835697,5.31%
2020,870517,0,1276,15504,853737,1.93%
2021,992295,4591,1246,15702,970756,2.17%
2022,995441,0,1045,11960,982436,1.31%
2023,1022060,0,621,16448,1004991,1.67%
2024,1061605,0,462,7092,1054051,0.71%


### ⚡ Xử lý Dữ liệu Vector hóa (Tối ưu hóa Hiệu năng)

1.  **Cấu hình Tham số:**
    * `NGUONG_DIEM_SAN_TO_HOP = 12.5`: Thiết lập điểm sàn xét tuyển (chỉ quan tâm các tổ hợp có tổng điểm $\ge$ 15).
    * `STEP = 0.05`: Xác định bước nhảy của điểm số (độ mịn của lưới dữ liệu). Tham số này quan trọng để xây dựng các biểu đồ mật độ (Heatmap) hoặc lưới điểm chuẩn xác.

2.  **Thực thi (`process_files_vectorized`):**
    * Hàm này sẽ đọc dữ liệu thô, ánh xạ thông tin tỉnh thành (Geocoding), tính toán điểm tổ hợp và chuẩn hóa dữ liệu đầu ra theo dạng lưới (Grid) để phục vụ cho việc trực quan hóa địa lý.

In [5]:
import sys
import os

sys.path.append(os.path.abspath('..'))
from src.preprocessing_diem_thi import process_files_vectorized

NGUONG_DIEM_SAN_TO_HOP = 12.5
STEP = 0.05
process_files_vectorized(NGUONG_DIEM_SAN_TO_HOP, STEP)

Đang load dữ liệu tỉnh...

Đang xử lý file: ../data/diem_thi_2019_new.csv
Số dòng ban đầu: 835,697

Năm 2019: 835,697 thí sinh
  Khối A00: 326,517 thí sinh hợp lệ
  Khối A01: 317,556 thí sinh hợp lệ
  Khối A02: 322,786 thí sinh hợp lệ
  Khối A03: 28,939 thí sinh hợp lệ
  Khối A04: 28,921 thí sinh hợp lệ
  Khối A05: 28,944 thí sinh hợp lệ
  Khối A06: 28,921 thí sinh hợp lệ
  Khối A07: 522,221 thí sinh hợp lệ
  Khối A08: 481,585 thí sinh hợp lệ
  Khối A09: 481,583 thí sinh hợp lệ
  Khối A10: 26,102 thí sinh hợp lệ
  Khối A11: 26,096 thí sinh hợp lệ
  Khối B00: 327,252 thí sinh hợp lệ
  Khối B01: 28,906 thí sinh hợp lệ
  Khối B02: 28,895 thí sinh hợp lệ
  Khối B03: 321,070 thí sinh hợp lệ
  Khối B04: 26,093 thí sinh hợp lệ
  Khối B08: 316,437 thí sinh hợp lệ
  Khối C00: 525,410 thí sinh hợp lệ
  Khối C01: 321,228 thí sinh hợp lệ
  Khối C02: 321,195 thí sinh hợp lệ
  Khối C03: 526,491 thí sinh hợp lệ
  Khối C04: 522,176 thí sinh hợp lệ
  Khối C05: 321,090 thí sinh hợp lệ
  Khối C06: 320,98

d:\2025-2026-HKI\IntroduceData\Task_4\introduction-to-data-science\src\preprocessing_diem_thi.py:211: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Số dòng ban đầu: 970,756

Năm 2021: 970,756 thí sinh
  Khối A00: 342,198 thí sinh hợp lệ
  Khối A01: 331,954 thí sinh hợp lệ
  Khối A02: 337,571 thí sinh hợp lệ
  Khối A07: 606,354 thí sinh hợp lệ
  Khối A08: 529,294 thí sinh hợp lệ
  Khối A09: 529,288 thí sinh hợp lệ
  Khối B00: 340,655 thí sinh hợp lệ
  Khối B03: 335,098 thí sinh hợp lệ
  Khối B08: 330,206 thí sinh hợp lệ
  Khối C00: 615,452 thí sinh hợp lệ
  Khối C01: 335,268 thí sinh hợp lệ
  Khối C02: 335,192 thí sinh hợp lệ
  Khối C03: 610,794 thí sinh hợp lệ
  Khối C04: 606,388 thí sinh hợp lệ
  Khối C05: 335,143 thí sinh hợp lệ
  Khối C06: 335,066 thí sinh hợp lệ
  Khối C08: 335,090 thí sinh hợp lệ
  Khối C14: 529,319 thí sinh hợp lệ
  Khối C19: 529,627 thí sinh hợp lệ
  Khối C20: 529,661 thí sinh hợp lệ
  Khối D01: 862,013 thí sinh hợp lệ
  Khối D07: 330,754 thí sinh hợp lệ
  Khối D08: 330,206 thí sinh hợp lệ
  Khối D09: 528,697 thí sinh hợp lệ
  Khối D10: 528,602 thí sinh hợp lệ
  Khối D11: 329,720 thí sinh hợp lệ
  Khối D14:

### 🌍 Thu thập Dữ liệu Địa lý & Tọa độ Tỉnh thành

1.  **Khởi tạo Danh mục (`provinces`):**
    * Định nghĩa danh sách chuẩn bao gồm **Mã sở (Code)** và **Tên tỉnh thành** tương ứng. Danh sách này đóng vai trò là bảng tham chiếu chính (Master Data) để liên kết với dữ liệu điểm thi.

2.  **Lấy Tọa độ Địa lý (`geocode_province`):**
    * Sử dụng **Mapbox API** (thông qua `MAPBOX_TOKEN`) để tự động truy vấn và lấy thông tin **Kinh độ (Longitude)** và **Vĩ độ (Latitude)** cho từng tỉnh thành.
    * Kết quả sẽ được lưu lại (thường là file `.csv` hoặc `.json`) để không phải gọi lại API nhiều lần.

3.  **Thu thập Dữ liệu Bổ sung (`crawl_province`):**
    * Thực hiện thu thập (crawl) thêm các thông tin meta-data cần thiết khác của tỉnh thành (nếu có) để làm giàu bộ dữ liệu.

In [ ]:
# Lấy kinh độ, vĩ độ cho từng tỉnh thành
provinces = [
    {'code': '01', 'name': 'Hà Nội'},
    {'code': '02', 'name': 'Thành phố Hồ Chí Minh'},
    {'code': '03', 'name': 'Hải Phòng'},
    {'code': '04', 'name': 'Đà Nẵng'},
    {'code': '05', 'name': 'Hà Giang'},
    {'code': '06', 'name': 'Cao Bằng'},
    {'code': '07', 'name': 'Lai Châu'},
    {'code': '08', 'name': 'Lào Cai'},
    {'code': '09', 'name': 'Tuyên Quang'},
    {'code': '10', 'name': 'Lạng Sơn'},
    {'code': '11', 'name': 'Bắc Kạn'},
    {'code': '12', 'name': 'Thái Nguyên'},
    {'code': '13', 'name': 'Yên Bái'},
    {'code': '14', 'name': 'Sơn La'},
    {'code': '15', 'name': 'Phú Thọ'},
    {'code': '16', 'name': 'Vĩnh Phúc'},
    {'code': '17', 'name': 'Quảng Ninh'},
    {'code': '18', 'name': 'Bắc Giang'},
    {'code': '19', 'name': 'Bắc Ninh'},
    {'code': '21', 'name': 'Hải Dương'},
    {'code': '22', 'name': 'Hưng Yên'},
    {'code': '23', 'name': 'Hòa Bình'},
    {'code': '24', 'name': 'Hà Nam'},
    {'code': '25', 'name': 'Nam Định'},
    {'code': '26', 'name': 'Thái Bình'},
    {'code': '27', 'name': 'Ninh Bình'},
    {'code': '28', 'name': 'Thanh Hóa'},
    {'code': '29', 'name': 'Nghệ An'},
    {'code': '30', 'name': 'Hà Tĩnh'},
    {'code': '31', 'name': 'Quảng Bình'},
    {'code': '32', 'name': 'Quảng Trị'},
    {'code': '33', 'name': 'Thừa Thiên Huế'},
    {'code': '34', 'name': 'Quảng Nam'},
    {'code': '35', 'name': 'Quảng Ngãi'},
    {'code': '36', 'name': 'Kon Tum'},
    {'code': '37', 'name': 'Bình Định'},
    {'code': '38', 'name': 'Gia Lai'},
    {'code': '39', 'name': 'Phú Yên'},
    {'code': '40', 'name': 'Đắk Lắk'},
    {'code': '41', 'name': 'Khánh Hòa'},
    {'code': '42', 'name': 'Lâm Đồng'},
    {'code': '43', 'name': 'Bình Phước'},
    {'code': '44', 'name': 'Bình Dương'},
    {'code': '45', 'name': 'Ninh Thuận'},
    {'code': '46', 'name': 'Tây Ninh'},
    {'code': '47', 'name': 'Bình Thuận'},
    {'code': '48', 'name': 'Đồng Nai'},
    {'code': '49', 'name': 'Long An'},
    {'code': '50', 'name': 'Đồng Tháp'},
    {'code': '51', 'name': 'An Giang'},
    {'code': '52', 'name': 'Bà Rịa-Vũng Tàu'},
    {'code': '53', 'name': 'Tiền Giang'},
    {'code': '54', 'name': 'Kiên Giang'},
    {'code': '55', 'name': 'Cần Thơ'},
    {'code': '56', 'name': 'Bến Tre'},
    {'code': '57', 'name': 'Vĩnh Long'},
    {'code': '58', 'name': 'Trà Vinh'},
    {'code': '59', 'name': 'Sóc Trăng'},
    {'code': '60', 'name': 'Bạc Liêu'},
    {'code': '61', 'name': 'Cà Mau'},
    {'code': '62', 'name': 'Điện Biên'},
    {'code': '63', 'name': 'Đắk Nông'},
    {'code': '64', 'name': 'Hậu Giang'}
]

geocode_province(provinces, MAPBOX_TOKEN)
crawl_province()

pk.eyJ1IjoidHJ1b25nMTEwNHoiLCJhIjoiY21pb2hzbXMxMDF5bDNrcTNhd2F3d28xayJ9.kAM8eS6y5ZtUid-qOibFDg
Lỗi khi lấy tọa độ cho [{'code': '01', 'name': 'Hà Nội'}, {'code': '02', 'name': 'Thành phố Hồ Chí Minh'}, {'code': '03', 'name': 'Hải Phòng'}, {'code': '04', 'name': 'Đà Nẵng'}, {'code': '05', 'name': 'Hà Giang'}, {'code': '06', 'name': 'Cao Bằng'}, {'code': '07', 'name': 'Lai Châu'}, {'code': '08', 'name': 'Lào Cai'}, {'code': '09', 'name': 'Tuyên Quang'}, {'code': '10', 'name': 'Lạng Sơn'}, {'code': '11', 'name': 'Bắc Kạn'}, {'code': '12', 'name': 'Thái Nguyên'}, {'code': '13', 'name': 'Yên Bái'}, {'code': '14', 'name': 'Sơn La'}, {'code': '15', 'name': 'Phú Thọ'}, {'code': '16', 'name': 'Vĩnh Phúc'}, {'code': '17', 'name': 'Quảng Ninh'}, {'code': '18', 'name': 'Bắc Giang'}, {'code': '19', 'name': 'Bắc Ninh'}, {'code': '21', 'name': 'Hải Dương'}, {'code': '22', 'name': 'Hưng Yên'}, {'code': '23', 'name': 'Hòa Bình'}, {'code': '24', 'name': 'Hà Nam'}, {'code': '25', 'name': 'Nam Định'}, {'c

### 🎓 Chuẩn hóa Dữ liệu Điểm chuẩn Đại học

1.  **Thiết lập Môi trường Động:**
    * Sử dụng `pathlib` để tự động xác định `PROJECT_ROOT` và thêm vào `sys.path`. Điều này giúp code chạy ổn định trên mọi hệ điều hành (Windows/Linux/Mac) mà không gặp lỗi đường dẫn.

2.  **Định nghĩa Luồng Dữ liệu (Input/Output):**
    * **Input:** `diem_chuan_all.csv` (Dữ liệu thô tổng hợp).
    * **Output:** `diem_chuan_chuan_hoa.csv` (Dữ liệu sạch đã được chuẩn hóa tên ngành, mã ngành).
    * **Removed:** `diem_chuan_bi_loai.csv` (Lưu trữ các ngành bị loại bỏ do không đủ dữ liệu lịch sử để đảm bảo tính nhất quán cho mô hình).

3.  **Kích hoạt Xử lý (`process_diem_chuan`):**
    * Gọi hàm xử lý chính từ thư mục `src` để thực hiện các logic nghiệp vụ: chuẩn hóa tên ngành, xử lý mã ngành, và điền dữ liệu thiếu.

In [4]:
from pathlib import Path
import sys

current_dir = Path.cwd()
PROJECT_ROOT = current_dir.parent 

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

# 3. Thiết lập đường dẫn tới file Data (đi từ gốc vào folder 'data')
data_dir = PROJECT_ROOT / "data"

path_input = data_dir / "diem_chuan_all.csv"
path_output = data_dir / "diem_chuan_chuan_hoa.csv"
path_removed = data_dir / "diem_chuan_bi_loai.csv"

# 4. Kiểm tra và Chạy
if path_input.exists():
    try:
        from src.chuan_hoa_diem_chuan import process_diem_chuan
        print("Bắt đầu xử lý...")
        df_final = process_diem_chuan(path_input, path_output, path_removed)
        print("Xử lý thành công!")
        
    except ImportError as e:
        print(f"Không tìm thấy module. Chi tiết: {e}")
else:
    print(f"Lỗi: Không tìm thấy file csv tại {path_input}")

Bắt đầu xử lý...
Đang đọc dữ liệu từ: c:\Users\ADMIN\OneDrive - VNU-HCMUS\Documents\GitHub\introduction-to-data-science\data\diem_chuan_all.csv
Đang lọc các ngành có đủ dữ liệu từ 2019 đến 2024...
Hoàn tất! File lưu tại: c:\Users\ADMIN\OneDrive - VNU-HCMUS\Documents\GitHub\introduction-to-data-science\data\diem_chuan_chuan_hoa.csv
Xử lý thành công!
